In [3]:
import os
import csv
import platform
import pandas as pd
import networkx as nx
from graph_partitioning import GraphPartitioning, utils

pwd = %pwd



In [15]:
import random

def getNetworkMinMaxCoordinateLocations(path):
    minX = 1000000.0
    minY = 1000000.0
    maxX = -1000000.0
    maxY= -1000000.0
    
    with open(path, 'r+') as f:
        for line in f:
            line = line.strip()
            xy = line.split(',')
            if(len(xy) == 2):
                x = float(xy[0])
                y = float(xy[1])
                
                if x > maxX:
                    maxX = x
                if x < minX:
                    minX = x
                if y > maxY:
                    maxY = y
                if y < minY:
                    minY = y
    return [(minX, maxX), (minY, maxY)]

def generateNodeCoordinates(xRange, yRange, numCoordinates):
    coords = []
    for i in range(0, numCoordinates):
        # generate x, y
        x = random.uniform(xRange[0], xRange[1])
        y = random.uniform(yRange[0], yRange[1])
        coords.append((x, y))
    return coords
        
    
import tempfile
import shutil

class LonelyNodesModifier:
    def __init__(self, graphPartitioner, percentageToAdd, additionLocation, isEnabled):
        self.tmpDir = tempfile.mkdtemp()
        self.gp = graphPartitioner
        self.isEnabled = isEnabled
        self.additionLocation = additionLocation
        self.percentageToAdd = percentageToAdd

    def add_lonely_nodes(self):
        if self.isEnabled:
            # update graph
            self.numLonelyNodesAdded = utils.add_lonely_nodes_to_graph(self.gp.G, self.percentageToAdd, self.additionLocation)

            # update simulated arrivals list
            self.gp.simulated_arrival_list = utils.add_lonely_node_arrivals(self.gp.simulated_arrival_list, self.numLonelyNodesAdded, self.additionLocation)

            # update fixed
        
    def setLonelyCoordinates(self):
        if self.isEnabled:
            # get coordinate locations
            self.xRange, self.yRange = getNetworkMinMaxCoordinateLocations(self.gp.COORDINATE_FILE)
            coords = generateNodeCoordinates(self.xRange, self.yRange, self.numLonelyNodesAdded)

                                
            # save coordinate file to right location
            self.coordPath = os.path.join(self.tmpDir, 'COORDINATES.csv')
            with open(self.coordPath, 'w') as outF:
                if self.additionLocation == 'before':
                    self._writeCoordsToFileStream(coords, outF)
                with open(self.gp.COORDINATE_FILE) as inF:
                    for line in inF:
                        outF.write(line)
                if self.additionLocation == 'after':
                    self._writeCoordsToFileStream(coords, outF)
                
    def _writeCoordToFileAsLine(self, coordinate, fStream):
            s = str(coordinate[0]) + ',' + str(coordinate[1]) + '\n'
            fStream.write(s)

    def _writeCoordsToFileStream(self, coordinates, fStream):
        for coord in coordinates:
            self._writeCoordToFileAsLine(coord, fStream)
            
    def __del__(self):
        if os.path.exists(self.tmpDir):
            shutil.rmtree(self.tmpDir)


In [19]:

class GP:
    def __init__(self):
        self.graphPath = os.path.join(pwd, "data", "predition_model_tests", "network", "network_1.txt")
        self.SIMULATED_ARRIVAL_FILE = os.path.join(pwd,"data","predition_model_tests","dataset_1_shift_rotate","simulated_arrival_list","percentage_of_prediction_correct_100","arrival_100_1.txt")
        self.COORDINATE_FILE = os.path.join(pwd,"data","predition_model_tests","coordinates","coordinates_1.txt")

        self.G = utils.read_metis(graphPath)

        self.simulated_arrival_list = []
        with open(self.SIMULATED_ARRIVAL_FILE, "r") as ar:
            self.simulated_arrival_list = [int(line.rstrip('\n')) for line in ar]


        
gp = GP()

lm = LonelyNodesModifier(gp, 0.2, 'after', True)
lm.add_lonely_nodes()
lm.setLonelyCoordinates()


print(gp.G.number_of_nodes(), len(gp.simulated_arrival_list))
print(lm.coordPath)

1199 1200
/var/folders/b4/hlv7g_9x5hg2n0n422577ts00000gn/T/tmpuzvdnh2s/COORDINATES.csv


In [18]:



# load graph
G = utils.read_metis(graphPath)

# modify graph with lonely nodes
numLonelyNodesAdded = utils.add_lonely_nodes_to_graph(G, 0.2, 'after')

# load arrivals list
simulated_arrival_list = []
with open(SIMULATED_ARRIVAL_FILE, "r") as ar:
    simulated_arrival_list = [int(line.rstrip('\n')) for line in ar]

# modify arrivals list
simulated_arrival_list = utils.add_lonely_node_arrivals(simulated_arrival_list, numLonelyNodesAdded, 'before')

#print(simulated_arrival_list)

xRange, yRange = getNetworkMinMaxCoordinateLocations(COORDINATE_FILE)
coords = generateNodeCoordinates(xRange, yRange, numLonelyNodesAdded)



NameError: name 'SIMULATED_ARRIVAL_FILE' is not defined